#### Events

* Regular customer arrives
    - If both servers are idle, regular customer begins service at the regular server.
    - If only one server is idle, that one is chosen.
    - If both are busy, regular customer enters the regular queue.
    - Schedule next regular customer arrival.
* Express customer arrives
    - If both servers are idle, express customer begins service at the express server.
    - If only one server is idle, that one is chosen.
    - If both are busy, express customer enters the express queue.
    - Schedule next express customer arrival.
* Regular customer departs
    - If regular queue is nonempty, the next regular customer begins service.
    - If regular queue is empty but express queue is nonempty, the next express customer begins service.
    - If both queues are empty, the server becomes idle.
* Express customer departs
    - If express queue is nonempty, the next express customer begins service.
    - If express queue is empty but regular queue is nonempty, the next regular customer begins service.
    - If both queues are empty, the server becomes idle.


In [1]:
import numpy as np
from dataclasses import dataclass

def expon(mean): 
    return np.random.exponential(scale=mean)

@dataclass
class Server:
    busy: bool = False

    @property
    def is_busy(self):
        return self.busy
    
    @property
    def is_idle(self):
        return not self.busy
    
from enum import Enum

class CType(Enum):
    REGULAR = 0
    EXPRESS = 1

@dataclass
class Customer:
    arrival_time: float
    type_: CType = CType.REGULAR

SERVER_REGULAR = Server()
SERVER_EXPRESS = Server()

In [61]:
MEAN_ARRIVAL_REGULAR = 2.1
MEAN_ARRIVAL_EXPRESS = 1.1
MEAN_SERVICE_REGULAR = 2.0
MEAN_SERVICE_EXPRESS = 0.9


def initialize():
    global SIM_CLOCK, EVENT_LIST, MEAN_ARRIVAL_EXPRESS, MEAN_ARRIVAL_REGULAR
    global TIME_LAST_EVENT
    SIM_CLOCK = 0.0
    TIME_LAST_EVENT = 0.0

    EVENT_LIST = [
        SIM_CLOCK + expon(MEAN_ARRIVAL_REGULAR), # regular customer arrives
        SIM_CLOCK + expon(MEAN_ARRIVAL_EXPRESS), # express customer arrives
        np.inf, # regular customer departs
        np.inf, # express customer departs
        60*8.0, # termination time (8 hours)
    ]

    # initialize servers
    global SERVER_EXPRESS, SERVER_REGULAR
    SERVER_REGULAR, SERVER_EXPRESS = Server(), Server()

    # initialize queues
    global Q_REGULAR, Q_EXPRESS
    Q_REGULAR, Q_EXPRESS = [], []

    # stats
    global NUM_CUSTS_EXPRESS, NUM_CUSTS_REGULAR
    NUM_CUSTS_EXPRESS = 0
    NUM_CUSTS_REGULAR = 0
    global TOTAL_DELAYS_EXPRESS, TOTAL_DELAYS_REGULAR
    TOTAL_DELAYS_EXPRESS = 0.0
    TOTAL_DELAYS_REGULAR = 0.0

    global AREA_NUM_IN_Q_EXPRESS, AREA_NUM_IN_Q_REGULAR
    AREA_NUM_IN_Q_REGULAR = 0.0
    AREA_NUM_IN_Q_EXPRESS = 0.0

    global AREA_SERVER_STATUS_REGULAR, AREA_SERVER_STATUS_EXPRESS
    AREA_SERVER_STATUS_REGULAR = 0.0
    AREA_SERVER_STATUS_EXPRESS = 0.0


def timing():
    global EVENT_LIST, NEXT_EVENT_TYPE, SIM_CLOCK
    # find most imminent event
    event_idx = np.argmin(EVENT_LIST)

    # record next event type
    NEXT_EVENT_TYPE = event_idx

    # advance sim clock
    SIM_CLOCK = EVENT_LIST[event_idx]


def regular_customer_arrives():
    # 1. state changes are made
    # 2. future events are scheduled
    # 3. event is removed from event list
    global SIM_CLOCK
    customer = Customer(arrival_time=SIM_CLOCK, type_=CType.REGULAR)

    global SERVER_EXPRESS, SERVER_REGULAR
    global Q_REGULAR, Q_EXPRESS
    global EVENT_LIST
    global NUM_CUSTS_REGULAR
    global TOTAL_DELAYS_REGULAR
    if SERVER_REGULAR.is_idle:
        # begin service at the regular server
        delay = SIM_CLOCK - customer.arrival_time # 0.0s
        TOTAL_DELAYS_REGULAR += delay
        NUM_CUSTS_REGULAR += 1
        SERVER_REGULAR.busy = True
        # schedule departure
        EVENT_LIST[2] = SIM_CLOCK + expon(MEAN_SERVICE_REGULAR)
    
    elif SERVER_REGULAR.is_busy and SERVER_EXPRESS.is_idle:
        # begin service at the express server
        delay = SIM_CLOCK - customer.arrival_time # 0.0s
        TOTAL_DELAYS_REGULAR += delay
        NUM_CUSTS_REGULAR += 1
        SERVER_EXPRESS.busy = True
        # schedule departure
        EVENT_LIST[3] = SIM_CLOCK + expon(MEAN_SERVICE_REGULAR)

    else: # both are busy
        # regular customer enters the regular queue
        Q_REGULAR.append(customer)
    
    # schedule next regular arrival
    global MEAN_ARRIVAL_REGULAR
    EVENT_LIST[0] = SIM_CLOCK + expon(MEAN_ARRIVAL_REGULAR)

def regular_customer_departs():
    global SIM_CLOCK
    global SERVER_REGULAR, Q_EXPRESS, Q_REGULAR
    global TOTAL_DELAYS_EXPRESS, TOTAL_DELAYS_REGULAR
    global NUM_CUSTS_REGULAR, NUM_CUSTS_EXPRESS

    if not Q_REGULAR and not Q_EXPRESS:
        # both queues are empty
        SERVER_REGULAR.busy = False
        EVENT_LIST[2] = np.inf # futher departures are impossible
    elif Q_REGULAR:
        # there is a customer in the regular queue, so begin service
        customer = Q_REGULAR[0]
        Q_REGULAR = Q_REGULAR[1:]
        delay = SIM_CLOCK - customer.arrival_time
        TOTAL_DELAYS_REGULAR += delay
        NUM_CUSTS_REGULAR += 1
        # schedule departure
        global MEAN_SERVICE_REGULAR
        EVENT_LIST[2] = SIM_CLOCK + expon(MEAN_SERVICE_REGULAR)
    else:
        # there is a customer in the express queue, so begin service
        customer = Q_EXPRESS[0]
        Q_EXPRESS = Q_EXPRESS[1:]
        delay = SIM_CLOCK - customer.arrival_time
        TOTAL_DELAYS_EXPRESS += delay
        NUM_CUSTS_EXPRESS += 1
        # schedule departure
        global MEAN_SERVICE_EXPRESS
        EVENT_LIST[2] = SIM_CLOCK + expon(MEAN_SERVICE_EXPRESS)


    

def express_customer_arrives():
    # 1. state changes are made
    # 2. future events are scheduled
    # 3. event is removed from event list
    global SIM_CLOCK
    customer = Customer(arrival_time=SIM_CLOCK, type_=CType.EXPRESS)

    global SERVER_EXPRESS, SERVER_REGULAR
    global Q_REGULAR, Q_EXPRESS
    global EVENT_LIST
    global NUM_CUSTS_EXPRESS
    global TOTAL_DELAYS_EXPRESS
    if SERVER_EXPRESS.is_idle:
        # begin service at the express server
        delay = SIM_CLOCK - customer.arrival_time # 0.0s
        SERVER_EXPRESS.busy = True
        TOTAL_DELAYS_EXPRESS += delay
        NUM_CUSTS_EXPRESS += 1
        # schedule departure
        EVENT_LIST[3] = SIM_CLOCK + expon(MEAN_SERVICE_EXPRESS)
    
    elif SERVER_REGULAR.is_idle and SERVER_EXPRESS.is_busy:
        # begin service at the regular server
        delay = SIM_CLOCK - customer.arrival_time # 0.0s
        TOTAL_DELAYS_EXPRESS += delay
        NUM_CUSTS_EXPRESS += 1
        SERVER_REGULAR.busy = True
        # schedule departure
        EVENT_LIST[2] = SIM_CLOCK + expon(MEAN_SERVICE_EXPRESS)

    else: # both are busy
        # express customer enters the express queue
        Q_EXPRESS.append(customer)
    
    # schedule next express arrival
    global MEAN_ARRIVAL_EXPRESS
    EVENT_LIST[1] = SIM_CLOCK + expon(MEAN_ARRIVAL_EXPRESS)

def express_customer_departs():
    global SIM_CLOCK
    global SERVER_REGULAR, Q_EXPRESS, Q_REGULAR

    global TOTAL_DELAYS_EXPRESS, TOTAL_DELAYS_REGULAR
    global NUM_CUSTS_EXPRESS, NUM_CUSTS_REGULAR
    if not Q_REGULAR and not Q_EXPRESS:
        # both queues are empty
        SERVER_EXPRESS.busy = False
        EVENT_LIST[3] = np.inf
    elif Q_EXPRESS:
        # there is a customer in the express queue, so begin service
        customer = Q_EXPRESS[0]
        Q_EXPRESS = Q_EXPRESS[1:]
        delay = SIM_CLOCK - customer.arrival_time
        TOTAL_DELAYS_EXPRESS += delay
        NUM_CUSTS_EXPRESS += 1
        # schedule departure
        global MEAN_SERVICE_EXPRESS
        EVENT_LIST[3] = SIM_CLOCK + expon(MEAN_SERVICE_EXPRESS)
    else:
        # there is a customer in the regular queue, so begin service
        customer = Q_REGULAR[0]
        Q_REGULAR = Q_REGULAR[1:]
        delay = SIM_CLOCK - customer.arrival_time
        TOTAL_DELAYS_REGULAR += delay
        NUM_CUSTS_REGULAR += 1
        # schedule departure
        global MEAN_SERVICE_REGULAR
        EVENT_LIST[3] = SIM_CLOCK + expon(MEAN_SERVICE_REGULAR)

def update_stats():
    global SIM_CLOCK, TIME_LAST_EVENT
    time_since_last_event = SIM_CLOCK - TIME_LAST_EVENT
    TIME_LAST_EVENT = SIM_CLOCK

    # update area under number-in-queue function
    global AREA_NUM_IN_Q_REGULAR, Q_REGULAR
    AREA_NUM_IN_Q_REGULAR += len(Q_REGULAR) * time_since_last_event
    # update area under number-in-queue function
    global AREA_NUM_IN_Q_EXPRESS, Q_EXPRESS
    AREA_NUM_IN_Q_EXPRESS += len(Q_EXPRESS) * time_since_last_event

    # update area under server status function
    global AREA_SERVER_STATUS_REGULAR, AREA_SERVER_STATUS_EXPRESS
    global SERVER_EXPRESS, SERVER_REGULAR
    if SERVER_REGULAR.busy:
        AREA_SERVER_STATUS_REGULAR += 1.0 * time_since_last_event
    if SERVER_EXPRESS.busy:
        AREA_SERVER_STATUS_EXPRESS += 1.0 * time_since_last_event


def report():
    global TOTAL_DELAYS_EXPRESS, TOTAL_DELAYS_REGULAR
    global NUM_CUSTS_EXPRESS, NUM_CUSTS_REGULAR
    global AREA_NUM_IN_Q_EXPRESS, AREA_NUM_IN_Q_REGULAR, SIM_CLOCK
    global AREA_SERVER_STATUS_EXPRESS, AREA_SERVER_STATUS_REGULAR
    print(f"Average delay in express queue:  {TOTAL_DELAYS_EXPRESS / NUM_CUSTS_EXPRESS:10.2f}")
    print(f"Average number in express queue: {AREA_NUM_IN_Q_EXPRESS / SIM_CLOCK:10.3f}")
    print(f"Average express server utilization: {AREA_SERVER_STATUS_EXPRESS / SIM_CLOCK:10.3f}")
    print(f"\nAverage delay in regular queue:  {TOTAL_DELAYS_REGULAR / NUM_CUSTS_REGULAR:10.2f}")
    print(f"Average number in regular queue: {AREA_NUM_IN_Q_REGULAR / SIM_CLOCK:10.3f}")
    print(f"Average regular server utilization: {AREA_SERVER_STATUS_REGULAR / SIM_CLOCK:10.3f}")


def main():
    initialize()

    while True:
        timing() # determine next event; advance sim clock

        update_stats() # update statistical counters

        # invoke the next event function
        if NEXT_EVENT_TYPE == 0:
            regular_customer_arrives()
        elif NEXT_EVENT_TYPE == 1:
            express_customer_arrives()
        elif NEXT_EVENT_TYPE == 2:
            regular_customer_departs()
        elif NEXT_EVENT_TYPE == 3:
            express_customer_departs()
        else:
            report()
            break

In [62]:
main()

Average delay in express queue:        1.70
Average number in express queue:      1.523
Average express server utilization:      0.794

Average delay in regular queue:        3.25
Average number in regular queue:      1.448
Average regular server utilization:      0.812
